# Introduction to numerical robotics

This notebook is a very general introduction to Pinocchio. It presents the main method to manipulate the geometry model of a manipulator robot: set the configuration, compute the position of an effector, check the collision or the distance to the obstacle. The main idea is to give a brief introduction of the general topic: how to discover and learn a robot movement constrained by the environment, using iterative optimization methods.


In [ ]:
import magic_donotload

## Set up

For this class, we need the UR5 robot model (urdf, in the robotpkg_example_robot_data), the pinocchio python software, the python optimizers from scipy and for the display of the results: the gepetto viewer and the python matplotlib. We have that with this set of imports:

In [ ]:
import pinocchio as pin
import time
import numpy as np
from numpy.linalg import inv,norm,pinv,svd,eig
import matplotlib.pylab as plt
from scipy.optimize import fmin_bfgs, fmin_slsqp
from tp0.load_environment import createRobotWithObstacles,Target

Let's first load the robot model and display it. You should first start Gepetto Viewer: in a shell (CTRL-ALT-T to open a terminal), run the command: gepetto-gui. A new GUI widow will open (with the logo of our team): this is the viewer. You are not suppose to close this window during the rest of the class, and if you do close it, remember to reopen it before trying to display anything from Python.

For this class, I wrapped the methods to load the robot model and create the obstacle field.

In [ ]:
robot = createRobotWithObstacles()

The next few lines initialize a 3D viewer.

In [ ]:
#Viewer = pin.visualize.GepettoVisualizer
Viewer = pin.visualize.MeshcatVisualizer
viz = Viewer(robot.model, robot.collision_model, robot.visual_model)
viz.initViewer(loadModel=True)
viz.display(robot.q0)
hasattr(viz.viewer, 'jupyter_cell') and viz.viewer.jupyter_cell()

The scene should not be displayed in the viewer (check it). The robot and the red obstacles are encoded in the robot object (we will not need to see in depth what is inside this object). The object Target is the green dot that the robot should reach. You can change the target position by editing target.position, and display the new position with target.display().

You can display a new configuration of the robot with viz.display (take a numpy.array of dimension 6 in input):

In [ ]:
viz.display(np.array([0,1.,-1.5,0,0,0]))

We also set up a target with is visualized as a green dot:

In [ ]:
target = Target(viz,position = np.array([.5,.5]))

## Using the robot model
The robot is a 6 degrees-of-freedom (dof) manipulator. To make the example simple, we will only use the joints 1 and 2. We will then typically represent a configuration of the robot by a vector q2 of dimension 2. Yet the robot model is dimension 6, so let's define 2 helper functions to pass from 2 to 6 and from 6 to 2.


In [ ]:
# %load -r 17-27 tp0/solution.py

def q2_to_q6(q2):
     '''
     Transform a vector 2 into a vector 6, corresponding to locking 4 joints of the 6-dof arm. 
     '''
     q6 = np.zeros(6)
     q6.flat[[1,2]] = q2
     return q6

def q6_to_q2(q6):
     return q6[1:3]

The following function computes the position of the end effector (in 2d):

In [ ]:
# %load -r 28-32 tp0/solution.py

def endef(q):
     '''Return the 2d position of the end effector.'''
     pin.forwardKinematics(robot.model,robot.data,q)
     return robot.data.oMi[-1].translation[[0,2]]

The next function computes the distance between the end effector and the target.

In [ ]:
# %load -r 33-37 tp0/solution.py

def dist(q):
     '''Return the distance between the end effector end the target (2d).'''
     return norm(endef(q)-target.position)


The last function checks if the robot is in collision, and returns True if a collision is detected.

In [ ]:
# %load -r 37-42 tp0/solution.py

def coll(q):
     '''Return true if in collision, false otherwise.'''
     pin.updateGeometryPlacements(robot.model,robot.data,robot.collision_model,robot.collision_data,q)
     return pin.computeCollisions(robot.collision_model,robot.collision_data,False)


## Random search of a valid configuration
The free space is difficult to represent explicitely. We can sample the configuration space until a free configuration is found:

In [ ]:
def qrand(check=False):
    '''
    Return a random configuration. If check is True, this
    configuration is not in collision
    '''
    while True:
        q = q2_to_q6(np.random.rand(2)*6-3)  # sample between -3 and +3.
        if not check or not coll(q): return q

viz.display(qrand(check=True))

Let's now find a valid configuration that is arbitrarily close to the target: sample until dist is small enough and coll is false (you may want to display the random trials inside the loop).

In [ ]:
# %load -r 63-74 tp0/solution.py

# Sample a random free configuration where dist is small enough.
def qrandTarget(threshold=5e-2, display=True):
     while True:
          q = qrand()
          if display:
               viz.display(q)
               time.sleep(1e-3)
          if not coll(q) and dist(q)<threshold:
               return q
viz.display(qrandTarget())


## From a random configuration to the target
Let' s now start from a random configuration. How can we find a path that bring the robot toward the target without touching the obstacles. Any idea?

In [ ]:
%do_not_load -r 82-89 tp0/solution.py

## Configuration space
Let's try to have a better look of the configuration space. In this case, it is easy, as it is dimension 2: we can sample it exhaustively and plot it in 2d. For that, let's introduce another function to compute the distance to collision:

In [ ]:
# %load -r 51-59 tp0/solution.py

def collisionDistance(q):
     '''Return the minimal distance between robot and environment. '''
     threshold = 1e-2
     pin.updateGeometryPlacements(robot.model,robot.data,robot.collision_model,robot.collision_data,q)
     if pin.computeCollisions(robot.collision_model,robot.collision_data,False): return -threshold
     idx = pin.computeDistances(robot.collision_model,robot.collision_data)
     return pin.computeDistance(robot.collision_model,robot.collision_data,idx).min_distance - threshold


Now, let's sample the configuration space and plot the distance-to-target and the distance-to-obstacle field (I put 500 samples to spare your CPU, but we need at least 10x more for obtaining a good picture).

In [ ]:
# %load -r 95-129 tp0/solution.py

def sampleSpace(nbSamples=500):
     '''
     Sample nbSamples configurations and store them in two lists depending
     if the configuration is in free space (hfree) or in collision (hcol), along
     with the distance to the target and the distance to the obstacles.
     '''
     hcol = []
     hfree = []
     for i in range(nbSamples):
          q = qrand(False)
          if not coll(q):
               hfree.append( list(q6_to_q2(q).flat) + [ dist(q), collisionDistance(q) ])
          else:
               hcol.append(  list(q6_to_q2(q).flat) + [ dist(q), 1e-2 ])
     return hcol,hfree

def plotConfigurationSpace(hcol,hfree):
     '''
     Plot 2 "scatter" plots: the first one plot the distance to the target for 
     each configuration, the second plots the distance to the obstacles (axis q1,q2, 
     distance in the color space).
     '''
     htotal = hcol + hfree
     h=np.array(htotal)
     plt.subplot(2,1,1)
     plt.scatter(h[:,0],h[:,1],c=h[:,2],s=20,lw=0)
     plt.title("Distance to the target")
     plt.subplot(2,1,2)
     plt.scatter(h[:,0],h[:,1],c=h[:,3],s=20,lw=0)
     plt.title("Distance to the obstacles")

hcol,hfree = sampleSpace(100)
plotConfigurationSpace(hcol,hfree)


## Optimize the distance under non-collision constraint
Finally, let's use one of the classic python solvers (from scipy) to search for a robot configuration that optimizes the distance to the target, under the constraint that the distance to collision is positive.
For that, we need to define a cost function (taking the robot configuration and return a scalar) and a constraint function (taking again the robot configuration and returning a scalar or a vector of scalar that should be positive). We additionally use the "callback" functionnality of the solver, to render the robot configuration corresponding to the current value of the decision variable inside the solver algorithm.
We are going to use the "SLSQP" solver from scipy, which implements a "sequential quadratic program" algorithm and accepts constraints.

In [ ]:
# %load  -r 133- tp0/solution.py

def cost(q):
     '''
     Cost function: distance to the target
     '''
     return dist(q2_to_q6(q))
     
def constraint(q):
     '''
     Constraint function: distance to the obstacle should be positive.
     '''
     return collisionDistance(q2_to_q6(q))
     
def callback(q):
     '''
     At each optimization step, display the robot configuration in gepetto-viewer.
     '''
     viz.display(q2_to_q6(q))
     time.sleep(.01)

def optimize():
     '''
     Optimize from an initial random configuration to discover a collision-free
     configuration as close as possible to the target. 
     '''
     return fmin_slsqp(x0=qrand(check=True),
                       func=cost,
                       f_ieqcons=constraint,callback=callback)

optimize()


Look at the output of the solver. It always returns a variable value, but sometimes the algorithm fails being traped in an unfeasible region. Most of the time, the solver converges to a local minimum where the final distance to the target is nonzero